In [49]:
#abc de scrapeo con bs
#a: imports
import pandas as pd
import requests as req
from bs4 import BeautifulSoup as bs

#b: ingredientes
url = 'https://books.toscrape.com/'
html = req.get(url).text
soup = bs(html, 'html.parser')
#soup

#c: extracción de elementos iterables dentro del html
pages = soup.find_all('a')

# encontrar la etiqueta 'a' de interés
print(len(pages)) # ---> 94
print(pages[93])

page = pages[93].attrs['href']
print(page) # ---> catalogue/page-2.html

books = soup.find_all('article',{'class':'product_pod'})
len(books) # ---> 20 libros correspondientes a cada pág


94
<a href="catalogue/page-2.html">next</a>
catalogue/page-2.html


20

In [59]:
#para navegar por cada página del catálogo:
#hacemos triquiñuela:
#creamos lista de páginas e iteramos, creando
pages_list = [url + f'catalogue/page-{n}.html' for n in range(1,6)]
pages_list

soups_list = []

for every_page in pages_list:
    
    single_page = req.get(every_page).text
    soupper = bs(single_page, 'html.parser')
    soups_list.append(soupper)

print(len(soups_list)) # ---> 5 (double-checked)
# soups_list[1] # ---> imprime el elemento dos de la lista de sopas correctamente
type(soups_list[0])

5


bs4.BeautifulSoup

In [119]:
# test = soups_list[0].find_all('a')
# test[54:94:2]
url


'https://books.toscrape.com/'

In [109]:
#listas para hacer mi DF a partir de diccionarios
titles_list = []
prices_list = []
links_list = []
pages_list = []

for every_book in soups_list:

    titles = every_book.find_all('h3')
    prices = every_book.find_all('p',{'class':'price_color'})
    links = every_book.find_all('a')

    for title in titles:
        titles_list.append(title.text)
    
    for price in prices:
        prices_list.append(price.text)

    for link in links[54:94:2]:
        links_list.append(link.attrs['href'])



In [123]:
#para acceder a toda la info de cada libro:
#creamos lista de links e iteramos, creando lista de sopas.
url_2 = url + 'catalogue/'
full_links_list = [url_2 + link for link in links_list]
soups_list2 = []


for link in full_links_list:

    single_html = req.get(link).text
    flavored_soup = bs(single_html, 'html.parser')
    soups_list2.append(flavored_soup)

len(soups_list2)

100

In [124]:
len(full_links_list), len(soups_list2)

full_links_list[1]

# https://books.toscrape.com/catalogue/tipping-the-velvet_999/index.html
# https://books.toscrape.com/catalogue/a-light-in-the-attic_1000/index.html

'https://books.toscrape.com/catalogue/tipping-the-velvet_999/index.html'

In [128]:
#más listas para hacer mi DF a partir de diccionarios
stock_list = []
covers_list = []
brief_descriptions_list = []
full_descriptions_list = []
upc_list = []
product_types_list = []
tax_list = []
num_reviews_list = []

for post in soups_list2:

    in_stock = post.find('p', {'class':'instock availability'}).text.strip()
    stock_list.append(in_stock)
    
    cover = post.find('img').attrs['src']
    covers_list.append(cover)

    brief_description = post.find_all('p')
    brief_descriptions_list.append(brief_description[3].text[:150])

    full_description = post.find_all('p')
    full_descriptions_list.append(full_description[3].text)

    upc = post.find('td').text
    upc_list.append(upc)

    product_type = post.find_all('td')
    product_types_list.append(product_type[1].text)

    tax = post.find_all('td')
    tax_list.append(tax[4].text)

    num_review = post.find_all('td')
    num_reviews_list.append(num_review[6].text)

print(len(stock_list)), print(len(covers_list)), print(len(brief_descriptions_list)), print(len(full_descriptions_list)), print(len(upc_list)), print(len(product_types_list)), print(len(tax_list)), print(len(num_reviews_list))

100
100
100
100
100
100
100
100


(None, None, None, None, None, None, None, None)

In [153]:
#finalmente, creamos DF a partir de lista de diccionarios
data = []

dicc_titles = {}
dicc_titles['book_title'] = titles_list
data.append(dicc_titles)

dicc_prices = {}
dicc_prices['price'] = prices_list
data.append(dicc_prices)

dicc_stock = {}
dicc_stock['availability'] = stock_list
data.append(dicc_stock)

dicc_covers = {}
dicc_covers['cover'] = covers_list
data.append(dicc_covers)

dicc_brief_descriptions = {}
dicc_prices['brief description'] = brief_descriptions_list
data.append(dicc_brief_descriptions)

dicc_full_descriptions = {}
dicc_full_descriptions['description'] = full_descriptions_list
data.append(dicc_prices)

dicc_upc = {}
dicc_upc['id'] = upc_list
data.append(dicc_upc)

dicc_product_types = {}
dicc_product_types['product type'] = product_types_list
data.append(dicc_product_types)

dicc_tax = {}
dicc_tax['tax'] = tax_list
data.append(dicc_tax)

dicc_num_reviews = {}
dicc_num_reviews['number of reviews'] = num_reviews_list
data.append(dicc_num_reviews)

dicc_unificado = {key: value for d in data for key, value in d.items()}
df = pd.DataFrame(dicc_unificado)
df

,book_title,price,brief description,availability,cover,id,product type,tax,number of reviews
0,A Light in the ...,Â£51.77,It's hard to imagine a world without A Light i...,In stock (22 available),../../media/cache/fe/72/fe72f0532301ec28892ae7...,a897fe39b1053632,Books,Â£0.00,0
1,Tipping the Velvet,Â£53.74,"""Erotic and absorbing...Written with starling ...",In stock (20 available),../../media/cache/08/e9/08e94f3731d7d6b760dfbf...,90fa61229261140a,Books,Â£0.00,0
2,Soumission,Â£50.10,"Dans une France assez proche de la nÃ´tre, un ...",In stock (20 available),../../media/cache/ee/cf/eecfe998905e455df12064...,6957f44c3847a760,Books,Â£0.00,0
3,Sharp Objects,Â£47.82,"WICKED above her hipbone, GIRL across her hear...",In stock (20 available),../../media/cache/c0/59/c05972805aa7201171b8fc...,e00eb4fd7b871a48,Books,Â£0.00,0
4,Sapiens: A Brief History ...,Â£54.23,From a renowned historian comes a groundbreaki...,In stock (20 available),../../media/cache/ce/5f/ce5f052c65cc963cf4422b...,4165285e1663650f,Books,Â£0.00,0
...,...,...,...,...,...,...,...,...,...
95,Lumberjanes Vol. 3: A ...,Â£19.92,This New York Times Bestselling series continu...,In stock (16 available),../../media/cache/d6/9b/d69b7020c4210e12882ffc...,57e5a4d515c8ce44,Books,Â£0.00,0
96,"Layered: Baking, Building, and ...",Â£40.11,Itâs time to venture beyond vanilla and choc...,In stock (16 available),../../media/cache/73/06/7306b860c7887dfb2a4b00...,7b75cc7f223791f4,Books,Â£0.00,0
97,Judo: Seven Steps to ...,Â£53.90,"Displaying the most impressive throws, compell...",In stock (16 available),../../media/cache/6d/80/6d80608ef81001ad068df0...,228f74b74f3a08ae,Books,Â£0.00,0
98,Join,Â£35.67,What if you could live multiple lives simultan...,In stock (16 available),../../media/cache/df/64/df642e483f7ae23de6888c...,18ad3ed896c7ce05,Books,Â£0.00,0


In [154]:
df['price'] = df['price'].str[1:]
df['tax'] = df['tax'].str[1:]
df.head(3)

,book_title,price,brief description,availability,cover,id,product type,tax,number of reviews
0,A Light in the ...,£51.77,It's hard to imagine a world without A Light i...,In stock (22 available),../../media/cache/fe/72/fe72f0532301ec28892ae7...,a897fe39b1053632,Books,£0.00,0
1,Tipping the Velvet,£53.74,"""Erotic and absorbing...Written with starling ...",In stock (20 available),../../media/cache/08/e9/08e94f3731d7d6b760dfbf...,90fa61229261140a,Books,£0.00,0
2,Soumission,£50.10,"Dans une France assez proche de la nÃ´tre, un ...",In stock (20 available),../../media/cache/ee/cf/eecfe998905e455df12064...,6957f44c3847a760,Books,£0.00,0
